# Q-learning in the worker search model

In this problem, we implement the Q-learning algorithm in the context of the McCall (1970) model. The problem is based on the corresponding <a href="https://python.quantecon.org/mccall_q.html">QuantEcon lecture</a>.


## Worker search model

The worker lives in an infinite-horizon economy, with discrete time \thinspace $t=0,1,2,\ldots $. Every period $t$, an iid wage offer $w$ from distribution $F\left(w\right) $ is drawn, with $F\left( 0\right) =0$, $F\left( B\right) =1$ for some $B>0$.

The worker decides to {accept or reject} the offer, $a_{t}\in \left\{ \text{accept, reject}\right\} $. Acceptance means that the worker receives income $y_{t}=w$ forever. Rejection implies that  the worker receives unemployment benefit $y_{t}=c$ and moves to next period where a new offer is drawn. Time is discounted at rate $\beta \in \lbrack 0,1)$.

The worker solves the sequence problem

\begin{equation}
V_{0}^{\ast }=\max_{\left\{ a_{t}\right\} _{t=0}^{\infty }}E_{0}\left[
\sum_{t=0}^{\infty }\beta ^{t}y_{t}\right]
\label{eq:mccall_infinite_horizon}
\end{equation}

where $a_{t}\in \left\{ \text{accept, reject}\right\} $ if the worker has
not yet accepted any earlier offer, and $a_{t}\in \left\{ {}\right\} $
otherwise. $V_{0}^{\ast }$ denotes the {value function}, and we assume that $V_{0}^{\ast }$
conditions on the initial offer $w_{0}$ being observed.

Every decision $a_{t}$ is made conditional on the time-$t$ information
set, which contains the history of all offers up to time $t$, $w^{t}=\left(
w_{0},\ldots ,w_{t}\right) $. $E\left[ \cdot \right] $ is the mathematical expectations operator

\begin{equation*}
E\left[ w\right] =\int_{0}^{B}wdF\left( w\right) =\int_{0}^{B}wf\left(
w\right) dw.
\end{equation*}


The of a worker with current offer $w$ at hand can
be formulated recursively as

\begin{equation*}
V\left( w\right) =\max_{\left\{ \text{accept, reject}\right\} }\left\{
V^{a}\left( w\right) ,c+\beta \int_{0}^{B}V\left( w^{\prime }\right)
dF\left( w^{\prime }\right) \right\}
\end{equation*}

where $V^{a}\left( w\right) $ is the value of accepting the offer.

In this formulation, we <b>assumed</b> that once an offer $w$ is accepted, the worker works at that wage
forever

\begin{equation*}
V^{a}\left( w\right) =\frac{w}{1-\beta }.
\end{equation*}

However, as it turns out, this assumption can be dropped without any consequence, and the problem modified as follows. In every period, the worker can decide whether to continue working at the same wage $w$ (i.e., accept the same wage offer $w$ again for the next period), or leave to unemployent, in which case a new wage offer arrives in the next period.

The value of accepting an offer can then be written as follows

\begin{equation*}
V^{a}\left( w\right) =w+\beta \max_{\left\{ \text{accept, reject}\right\}
}\left\{ V^{a}\left( w\right) ,c+\beta \int_{0}^{B}V\left( w^{\prime
}\right) dF\left( w^{\prime }\right) \right\}
\end{equation*}

In this iid environment, even if the worker is allowed to leave the current job that guarantees wage $w$ which was previously accepted, such an option would never be exercised. In other words, if the wage was sufficiently high to be accepted in one period, it is sufficiently high to be accepted in any other period.

Notice that this conclusion would be different in a model with an additional persistent state variable that affects the distribution of offers.

Also notice that this option to leave is distinct from <b>search on the job</b> where a worker samples new wage offers while continuing to work in the existing job, and only accepts an offer if it is better than the current wage.

## Q-learning

In the McCall (1970) model, the worker understands the probabilistic structure of the model, which allows to form expectations (subjective or objective) over the next-period offers.

We now envision a distinctly different algorithm. Imagine that the worker does not have available the probability distribution of next period offers. Instead, the worker observes realized draws and makes accept or reject decisions. The <b>Q-learning algorithm</b> is an example of <b>reinforcement learning algorithms</b> in which the worker is rewarded for making decisions that, over time, lead to high payoffs. Through this process, the worker learns the value of alternative actions in a given state, which then allows to deduce optimal action.

We start by rewriting the problem in the form of the <b>state-action value function</b> $Q\left(
w,a\right) $:

\begin{eqnarray*}
Q\left( {\color{#CC6677}{w}},\text{accept}\right) &=&{w}+\beta \max_{\left\{ \text{accept,
reject}\right\} }\left\{ Q\left( {w},\text{accept}\right) ,Q\left( {w},\text{reject}\right) \right\} \\
Q\left( {w},\text{reject}\right) &=&c+\beta \int_{0}^{B}\max_{\left\{ \text{accept, reject}\right\} }\left\{ Q\left( {w^\prime},\text{accept}\right)
,Q\left( {w^\prime},\text{reject}\right) \right\} dF\left( {w^\prime}\right)
\end{eqnarray*}

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
\begin{itemize}
\item notice the distinction between ${w}$ and ${w^\prime}$
\end{itemize}

We now have

\begin{equation*}
V^{a}\left( w\right) =Q\left( w,\text{accept}\right) \qquad V\left( w\right)
=\max_{\left\{ \text{accept, reject}\right\} }\left\{ Q\left( w,\text{accept}%
\right) ,Q\left( w,\text{reject}\right) \right\} .
\end{equation*}



## Theoretical exposition



% ==============================================================================

Temporal difference step

We can now replace the expectations with sample draws and form temporal
differences.

Start with an old iteration of the state-action value function $%
Q^{old}\left( w,a\right) $

\begin{eqnarray*}
TD\left( {w},\text{accept}\right) &=&{w}+\beta \max_{a^{\prime }\in \mathcal{%
A}}Q^{old}\left( {w},a^{\prime }\right) -Q^{old}\left( {w},\text{accept}%
\right) \\
TD\left( {w},\text{reject}\right) &=&c+\beta \max_{a^{\prime }\in \mathcal{A}%
}Q^{old}\left( {w^\prime},a\right) -Q^{old}\left( {w},\text{reject}\right)
\end{eqnarray*}%
for $\mathcal{A}=\left\{ \text{accept, reject}\right\} $ and ${w^\prime}\sim
F\left( {w^\prime}\right) $.

Then the new iteration of the state-action value function is%
\begin{equation*}
Q^{new}\left( w,a\right) =Q^{old}\left( w,a\right) +\alpha TD\left(
w,a\right)
\end{equation*}

% ==============================================================================

Implementation

{Discretization}

\begin{itemize}
\item action is already discrete, replace the state space $\left[ 0,B\right]
$ with a discrete grid $w^{i}$, $i=1,\ldots ,I$

\item replace continuous distribution $F\left( w\right) $ with a discrete
counterpart $\hat{f}^{i}$ of mass points on grid $w^{i}$

\item sample offers $w^{\prime }$ from $\hat{f}^{i}$, $i=1,\ldots ,I$

\item alternatively, use a form of projection and a `deep Q-learning'
algorithm to update the projection coefficients
\end{itemize}

{Experimentation}: modify the algorithm to incorporate deviations from
currently `optimal' choice

\begin{itemize}
\item in every step, with probability $\varepsilon $, replace $%
\max_{a^{\prime }\in \mathcal{A}}$ with $\min_{a^{\prime }\in \mathcal{A}}$

\item allow exploration of alternatives that may be omitted if algorithm
gets stuck in a local maximum
\end{itemize}

## Implementation

In [1]:
# import some useful predefined functions from the course package
from course import *

Done everything.
